In [ ]:
import pandas as pd
from MRNN.main_mrnn import *
# combined_data_lst = ['combined_oura.csv', 'user_581_survey_data.csv']
result_lst = []
for f in combined_data_lst:
# for f in os.listdir(os.path.join('.', 'dataset')):
    # if 'combine' in f:
    #     continue
    file_name = './dataset/' + f
    #file_name = './MRNN/data/google.csv'
    test_df = pd.read_csv(file_name)
    seq_len = 10
    # seq_len = 50
    h_dim = 10
    batch_size = 8
    iteration = 20
    learning_rate = 0.01
    metric_name = 'rmse'
    #0.1, 0.3, 0.5, 
    for missing_rate in [0.5]:
        result_lst.append(main({
            'file_name': file_name,
            'seq_len': seq_len,
            'missing_rate': missing_rate,
            'h_dim': h_dim,
            'batch_size': batch_size,
            'iteration': iteration,
            'learning_rate': learning_rate,
            'metric_name': metric_name
        }))
#   Args:
#     - file_name: dataset file name
#     - seq_len: sequence length of time-series data
#     - missing_rate: the rate of introduced missingness
#     - h_dim: hidden state dimensions
#     - batch_size: the number of samples in mini batch
#     - iteration: the number of iteration
#     - learning_rate: learning rate of model training
#     - metric_name: imputation performance metric (mse, mae, rmse)
    
# !cd ./MRNN
# !python3 ./MRNN/main_mrnn.py --file_name ./MRNN/data/google.csv --seq_len 7 --missing_rate 0.2 --h_dim 10 --batch_size 128 --iteration 2000 --learning_rate 0.01 --metric_name rmse

In [ ]:
from itertools import product
class GridSearcher:
    def __init__(self, param_grid):
        self.param_grid = [param_grid]
    
    def __iter__(self):
        """Iterate over the points in the grid.
        Returns
        -------
        params : iterator over dict of str to any
            Yields dictionaries mapping each estimator parameter to one of its
            allowed values.
        """
        for p in self.param_grid:
            # Always sort the keys of a dictionary, for reproducibility
            items = sorted(p.items())
            if not items:
                yield {}
            else:
                keys, values = zip(*items)
                for v in product(*values):
                    params = dict(zip(keys, v))
                    yield params
param_grid = {
    'seq_len': [5, 10, 20],
    'h_dim': [5, 10, 20],
    'batch_size': [8, 16, 32],
    'iteration': [50, 200, 500],
    'learning_rate': [0.01, 0.001]
}
my_searcher = GridSearcher(param_grid)
for g in my_searcher:
    print(g)

In [ ]:
import pandas as pd
from MRNN.main_mrnn import *
# combined_data_lst = ['combined_oura.csv', 'user_581_survey_data.csv']
param_grid_result_lst = []
for f in combined_data_lst:
# for f in os.listdir(os.path.join('.', 'dataset')):
    # if 'combine' in f:
    #     continue
    file_name = './dataset/' + f
    #file_name = './MRNN/data/google.csv'
    test_df = pd.read_csv(file_name)
    # seq_len = 10
    # # seq_len = 50
    # h_dim = 10
    # batch_size = 8
    # iteration = 20
    # learning_rate = 0.01
    metric_name = 'rmse'
    #0.1, 0.3, 0.5, 
    for parameters in my_searcher:
        for missing_rate in [0.1, 0.3, 0.5]:
            try:
                performance = main({
                    'file_name': file_name,
                    'seq_len': parameters['seq_len'],
                    'missing_rate': missing_rate,
                    'h_dim': parameters['h_dim'],
                    'batch_size': parameters['batch_size'],
                    'iteration': parameters['iteration'],
                    'learning_rate': parameters['learning_rate'],
                    'metric_name': metric_name
                })['performance']
            except:
                performance = np.nan
            param_grid_result_lst.append({
                'file_name': f,
                'seq_len': parameters['seq_len'],
                'missing_rate': missing_rate,
                'h_dim': parameters['h_dim'],
                'batch_size': parameters['batch_size'],
                'iteration': parameters['iteration'],
                'learning_rate': parameters['learning_rate'],
                'metric_name': metric_name,
                'rmse': performance
            })

In [ ]:
result_lst[0]['performance']

In [ ]:
import pickle
with open('param_grid_result_lst.pkl', 'wb') as f:
    pickle.dump(param_grid_result_lst, f)

In [ ]:
from hyperimpute.plugins.imputers import Imputers

In [ ]:
import pickle
import numpy as np
with open('param_grid_result_lst.pkl', 'rb') as f:
    results = pickle.load(f)
not_nan_results = []
for r in results:
    if not np.isnan(r['rmse']):
        not_nan_results.append(r)
print(len(not_nan_results))
print(not_nan_results[0].keys())




In [ ]:
missing_rate = 0.1
file_name='combined_oura.csv' # or "user_581_survey_data.csv"
file_name = "user_581_survey_data.csv"
# for r in not_nan_results:
#     if r['missing_rate'] == missing_rate:
#         print(type(r['rmse']), r['rmse'], np.isnan(r['rmse']))
filtered_results = [x for x in not_nan_results if x['missing_rate']==missing_rate and x['file_name']==file_name]
min(filtered_results, key=lambda x: x['rmse'])
